In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from itables import init_notebook_mode
from tensorflow.keras.preprocessing.image import ImageDataGenerator

init_notebook_mode(all_interactive=True)

required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

In [ ]:
DATASET_CSV = os.path.abspath("./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv")
DATASET_PATH = os.path.dirname(DATASET_CSV)

IMAGE_SIZE = (224, 224)

dataset_df = pd.read_csv(DATASET_CSV, sep=";")
dataset_df['file_name'] = dataset_df['file_name'].apply(lambda x: os.path.join(DATASET_PATH, x))

dataset_df['label'] = dataset_df['label'].astype(str)
dataset_df['binary_label'] = dataset_df['binary_label'].astype(str)
dataset_df['detector_label'] = dataset_df['detector_label'].astype(str)

train_dataframe = dataset_df[dataset_df['subset'] == "train"]
validation_dataframe = dataset_df[dataset_df['subset'] == "validation"]
test_dataframe = dataset_df[dataset_df['subset'] == "test"]

In [ ]:
IMAGE_SIZE = (224, 224)

data_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)
images = data_gen.flow_from_dataframe(
    dataframe=train_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",  # 🔥 Asegurar que las imágenes sean RGB (3 canales)
    class_mode="binary",
    batch_size=1,
    shuffle=False
)

data_augmentation = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    fill_mode="constant",
)
images_edited = data_augmentation.flow_from_dataframe(
    dataframe=train_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",  # 🔥 Asegurar que las imágenes sean RGB (3 canales)
    class_mode="binary",
    batch_size=1,
    shuffle=False
)


In [ ]:
INDEX = 3504

image_batch, label_batch = images[INDEX]
image = image_batch[0]
img = (image + 1.0) * 127.5
imagen_original = np.clip(img, 0, 255).astype(np.uint8)
label_original = int(label_batch[0])

image_batch, label_batch = images_edited[INDEX]
image = image_batch[0]
img = (image + 1.0) * 127.5
imagen_edited = np.clip(img, 0, 255).astype(np.uint8)
label_edited = int(label_batch[0])

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(imagen_original)
axes[0].set_title(f"Original Image, Class: {label_original}")
axes[0].axis("off")

axes[1].imshow(imagen_edited)
axes[1].set_title(f"Edited Image, Class: {label_edited}")
axes[1].axis("off")

plt.show()